In [1]:
!pip install -U sentence-transformers

  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/68/0d/27475988a3daade7516ea02dbc607b57d4a30f01bb49614a6430e76685c2/sentence_transformers-2.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.32.0 from https://files.pythonhosted.org/packages/b6/4d/fbe6d89fde59d8107f0a02816c4ac4542a8f9a85559fdf33c68282affcc1/transformers-4.38.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     --- -------------------------

In [3]:
pip install fasttext-wheel

  Obtaining dependency information for fasttext-wheel from https://files.pythonhosted.org/packages/96/58/2d1c2557cefa8d30c7e7ed182cac53cc811b4dcf265ffa64fb8e8a6287c5/fasttext_wheel-0.9.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/232.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.4 kB ? eta -:--:--
   ------ -------------------------------- 41.0/232.4 kB 245.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/232.4 kB 245.8 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/232.4 kB 272.3 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/232.4 kB 272.3 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/232.4 kB 275.8 kB/s eta 0:00:01
   ------------------ ------------------- 112.6/232.4 kB 284.4 kB/s eta 0:00:01
   -------------------- ----------------- 122.9/232.4 kB 276.8 kB/s eta 0:00:01
   

In [4]:
!pip install stanza

  Obtaining dependency information for stanza from https://files.pythonhosted.org/packages/03/3a/4554b9cd2218ff7d652406f27e957300b8fbae1700027367be2126196140/stanza-1.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/98/00/00d56e704d69cee4a92b1d517676579b4af5f2f8bc72946c464a504705b2/emoji-2.10.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.15.0 from https://files.pythonhosted.org/packages/ad/6e/1bed3b7c904cc178cb8ee8dbaf72934964452b3de95b7a63412591edb93c/protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/970.4 kB ? eta -:--:--
   - -------------------------------------- 30.7/970.4 kB ? eta -:--:--
   -- ------------------------------------ 61.4/970.4 kB 812.7 kB/s eta 0:00:02
   --- ----------------------------------- 92.2/970.4 kB 751.6 kB/s eta 0:00:02
   ----- -------------------------------- 143.4/970.4 kB 853.3 kB/s eta 0:00:01
  

In [5]:
import numpy as np

# Function to read the dataset
def read_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        # Create an empty list to store each row as a tuple (review, label)
        data = [(line.split('\t')[0], int(line.split('\t')[1].strip())) for line in lines]
    # Convert list of tuples into a NumPy array
    data_array = np.array(data, dtype=object)  # Using dtype=object to accommodate variable-length strings
    return data_array

def split_data(data_array, train_size=0.7, val_size=0.1, test_size=0.2):
    # Ensure the sizes sum up to 1
    assert (train_size + val_size + test_size) == 1, "The sizes must sum up to 1."

    # Shuffle the array
    np.random.shuffle(data_array)

    # Calculate split indices
    total_samples = len(data_array)
    train_end = int(total_samples * train_size)
    val_end = train_end + int(total_samples * val_size)

    # Split the dataset
    train_data = data_array[:train_end]
    val_data = data_array[train_end:val_end]
    test_data = data_array[val_end:]

    return train_data, val_data, test_data


file_path = 'yelp_labelled.txt'
dataset = read_dataset(file_path)

# Split the dataset
train, val, test = split_data(dataset)

# Example to check
print(f"Training set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")

Training set size: 700
Validation set size: 100
Test set size: 200


In [6]:
import fasttext

# Load the FastText model 
ft_model = fasttext.load_model('cc.en.300.bin')

In [7]:
def encode_reviews(reviews, model):
    # Encoding each review into a sentence vector
    vectors = [model.get_sentence_vector(review) for review in reviews]
    return np.array(vectors)

In [34]:
X_train = encode_reviews(train[:, 0], ft_model)
y_train = train[:, 1].astype('float')

X_val = encode_reviews(val[:, 0], ft_model)
y_val = val[:, 1].astype('float')

X_test = encode_reviews(test[:, 0], ft_model)
y_test = test[:, 1].astype('float')

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


penalties = ['l1', 'l2', 'elasticnet', None]
Cs = [0.001, 0.1, 1.0]
tols = [0.0001, 0.001, 0.01]
solver = 'saga'  # 'saga' supports all penalties

# Initialize variables to store the best model
best_accuracy = 0
best_params = {}

for penalty in penalties:
    for C in Cs:
        for tol in tols:
            # Elasticnet requires specifying l1_ratio; here, we use 0.5 as a default value.
            
            if penalty == 'elasticnet':
                l1_ratio = 0.5
            else:
                l1_ratio = None

            # Create and train the model
            model = LogisticRegression(penalty=penalty, C=C, tol=tol, solver=solver, l1_ratio=l1_ratio, max_iter=4000)
            try:
                model.fit(X_train, y_train)
                # Predict on the validation set
                y_val_pred = model.predict(X_val)
                # Calculate accuracy
                accuracy = accuracy_score(y_val, y_val_pred)
                print(f"Penalty: {penalty}, C: {C}, tol: {tol}, Accuracy: {accuracy}")

                # Update the best model if the current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'penalty': penalty, 'C': C, 'tol': tol, 'l1_ratio': l1_ratio}
            except ValueError as e:
                # Some configurations might be invalid depending on the solver, catch those errors
                print(f"Invalid configuration: Penalty: {penalty}, C: {C}, tol: {tol}, Error: {e}")

# Display the best configuration and its accuracy
print(f"Best Configuration: {best_params}, Best Validation Accuracy: {best_accuracy}")

Penalty: l1, C: 0.001, tol: 0.0001, Accuracy: 0.51
Penalty: l1, C: 0.001, tol: 0.001, Accuracy: 0.49
Penalty: l1, C: 0.001, tol: 0.01, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.0001, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.001, Accuracy: 0.51
Penalty: l1, C: 0.1, tol: 0.01, Accuracy: 0.49
Penalty: l1, C: 1.0, tol: 0.0001, Accuracy: 0.75
Penalty: l1, C: 1.0, tol: 0.001, Accuracy: 0.76
Penalty: l1, C: 1.0, tol: 0.01, Accuracy: 0.76
Penalty: l2, C: 0.001, tol: 0.0001, Accuracy: 0.49
Penalty: l2, C: 0.001, tol: 0.001, Accuracy: 0.49
Penalty: l2, C: 0.001, tol: 0.01, Accuracy: 0.49
Penalty: l2, C: 0.1, tol: 0.0001, Accuracy: 0.72
Penalty: l2, C: 0.1, tol: 0.001, Accuracy: 0.72
Penalty: l2, C: 0.1, tol: 0.01, Accuracy: 0.72
Penalty: l2, C: 1.0, tol: 0.0001, Accuracy: 0.77
Penalty: l2, C: 1.0, tol: 0.001, Accuracy: 0.77
Penalty: l2, C: 1.0, tol: 0.01, Accuracy: 0.77
Penalty: elasticnet, C: 0.001, tol: 0.0001, Accuracy: 0.49
Penalty: elasticnet, C: 0.001, tol: 0.001, Accuracy: 0.49
Penalty

C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.0001, Accuracy: 0.87


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.001, Accuracy: 0.85
Penalty: None, C: 0.001, tol: 0.01, Accuracy: 0.81


...

Penalty: None, C: 0.1, tol: 0.0001, Accuracy: 0.87


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.1, tol: 0.001, Accuracy: 0.85
Penalty: None, C: 0.1, tol: 0.01, Accuracy: 0.81


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 1.0, tol: 0.0001, Accuracy: 0.87
Penalty: None, C: 1.0, tol: 0.001, Accuracy: 0.85
Penalty: None, C: 1.0, tol: 0.01, Accuracy: 0.81
Best Configuration: {'penalty': None, 'C': 0.001, 'tol': 0.0001, 'l1_ratio': None}, Best Validation Accuracy: 0.87


In [48]:
best_model = LogisticRegression(penalty=best_params['penalty'], C=best_params['C'], tol=best_params['tol'], solver=solver, l1_ratio=best_params['l1_ratio'], max_iter=4000)
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy ont the test set with the best configuration is: ", accuracy)

C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Accuracy ont the test set with the best configuration is:  0.785


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False)

# Fit and transform the training data to compute TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(train[:,0])

# Transform the validation and test data (do not fit, to ensure consistency with the training data vocabulary)
X_val_tfidf = tfidf_vectorizer.transform(val[:,0])
X_test_tfidf = tfidf_vectorizer.transform(test[:,0])

In [36]:
for penalty in penalties:
    for C in Cs:
        for tol in tols:
            # Elasticnet requires specifying l1_ratio; here, we use 0.5 as a default value.
            
            if penalty == 'elasticnet':
                l1_ratio = 0.5
            else:
                l1_ratio = None

            # Create and train the model
            model = LogisticRegression(penalty=penalty, C=C, tol=tol, solver=solver, l1_ratio=l1_ratio, max_iter=4000)
            try:
                model.fit(X_train_tfidf, y_train)
                # Predict on the validation set
                y_val_pred = model.predict(X_val_tfidf)
                # Calculate accuracy
                accuracy = accuracy_score(y_val, y_val_pred)
                print(f"Penalty: {penalty}, C: {C}, tol: {tol}, Accuracy: {accuracy}")

                # Update the best model if the current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'penalty': penalty, 'C': C, 'tol': tol, 'l1_ratio': l1_ratio}
            except ValueError as e:
                # Some configurations might be invalid depending on the solver, catch those errors
                print(f"Invalid configuration: Penalty: {penalty}, C: {C}, tol: {tol}, Error: {e}")

# Display the best configuration and its accuracy
print(f"Best Configuration: {best_params}, Best Validation Accuracy: {best_accuracy}")

Penalty: l1, C: 0.001, tol: 0.0001, Accuracy: 0.51
Penalty: l1, C: 0.001, tol: 0.001, Accuracy: 0.49
Penalty: l1, C: 0.001, tol: 0.01, Accuracy: 0.51
Penalty: l1, C: 0.1, tol: 0.0001, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.001, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.01, Accuracy: 0.51
Penalty: l1, C: 1.0, tol: 0.0001, Accuracy: 0.75
Penalty: l1, C: 1.0, tol: 0.001, Accuracy: 0.75
Penalty: l1, C: 1.0, tol: 0.01, Accuracy: 0.74
Penalty: l2, C: 0.001, tol: 0.0001, Accuracy: 0.49
Penalty: l2, C: 0.001, tol: 0.001, Accuracy: 0.49
Penalty: l2, C: 0.001, tol: 0.01, Accuracy: 0.49
Penalty: l2, C: 0.1, tol: 0.0001, Accuracy: 0.77
Penalty: l2, C: 0.1, tol: 0.001, Accuracy: 0.77
Penalty: l2, C: 0.1, tol: 0.01, Accuracy: 0.77
Penalty: l2, C: 1.0, tol: 0.0001, Accuracy: 0.79
Penalty: l2, C: 1.0, tol: 0.001, Accuracy: 0.79
Penalty: l2, C: 1.0, tol: 0.01, Accuracy: 0.79
Penalty: elasticnet, C: 0.001, tol: 0.0001, Accuracy: 0.51
Penalty: elasticnet, C: 0.001, tol: 0.001, Accuracy: 0.51
Penalty

C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.0001, Accuracy: 0.82
Penalty: None, C: 0.001, tol: 0.001, Accuracy: 0.84
Penalty: None, C: 0.001, tol: 0.01, Accuracy: 0.85
Penalty: None, C: 0.1, tol: 0.0001, Accuracy: 0.82
Penalty: None, C: 0.1, tol: 0.001, Accuracy: 0.83
Penalty: None, C: 0.1, tol: 0.01, Accuracy: 0.82


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 1.0, tol: 0.0001, Accuracy: 0.81
Penalty: None, C: 1.0, tol: 0.001, Accuracy: 0.83
Penalty: None, C: 1.0, tol: 0.01, Accuracy: 0.85
Best Configuration: {'penalty': None, 'C': 0.001, 'tol': 0.0001, 'l1_ratio': None}, Best Validation Accuracy: 0.87


In [37]:
best_model = LogisticRegression(penalty=best_params['penalty'], C=best_params['C'], tol=best_params['tol'], solver=solver, l1_ratio=best_params['l1_ratio'], max_iter=4000)
best_model.fit(X_train_tfidf, y_train)
y_test_pred = best_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy ont the test set with the best configuration is: ", accuracy)

Accuracy ont the test set with the best configuration is:  0.77


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


In [38]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\iskak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iskak\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
X_train_embeddings = model.encode(train[:,0], batch_size=32, show_progress_bar=True)
X_val_embeddings = model.encode(val[:,0], batch_size=32, show_progress_bar=True)
X_test_embeddings = model.encode(test[:,0], batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [40]:
for penalty in penalties:
    for C in Cs:
        for tol in tols:
            # Elasticnet requires specifying l1_ratio; here, we use 0.5 as a default value.
            
            if penalty == 'elasticnet':
                l1_ratio = 0.5
            else:
                l1_ratio = None

            # Create and train the model
            model = LogisticRegression(penalty=penalty, C=C, tol=tol, solver=solver, l1_ratio=l1_ratio, max_iter=4000)
            try:
                model.fit(X_train_embeddings, y_train)
                # Predict on the validation set
                y_val_pred = model.predict(X_val_embeddings)
                # Calculate accuracy
                accuracy = accuracy_score(y_val, y_val_pred)
                print(f"Penalty: {penalty}, C: {C}, tol: {tol}, Accuracy: {accuracy}")

                # Update the best model if the current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'penalty': penalty, 'C': C, 'tol': tol, 'l1_ratio': l1_ratio}
            except ValueError as e:
                # Some configurations might be invalid depending on the solver, catch those errors
                print(f"Invalid configuration: Penalty: {penalty}, C: {C}, tol: {tol}, Error: {e}")

# Display the best configuration and its accuracy
print(f"Best Configuration: {best_params}, Best Validation Accuracy: {best_accuracy}")

Penalty: l1, C: 0.001, tol: 0.0001, Accuracy: 0.49
Penalty: l1, C: 0.001, tol: 0.001, Accuracy: 0.51
Penalty: l1, C: 0.001, tol: 0.01, Accuracy: 0.51
Penalty: l1, C: 0.1, tol: 0.0001, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.001, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.01, Accuracy: 0.49
Penalty: l1, C: 1.0, tol: 0.0001, Accuracy: 0.88
Penalty: l1, C: 1.0, tol: 0.001, Accuracy: 0.89
Penalty: l1, C: 1.0, tol: 0.01, Accuracy: 0.89
Penalty: l2, C: 0.001, tol: 0.0001, Accuracy: 0.51
Penalty: l2, C: 0.001, tol: 0.001, Accuracy: 0.51
Penalty: l2, C: 0.001, tol: 0.01, Accuracy: 0.5
Penalty: l2, C: 0.1, tol: 0.0001, Accuracy: 0.93
Penalty: l2, C: 0.1, tol: 0.001, Accuracy: 0.93
Penalty: l2, C: 0.1, tol: 0.01, Accuracy: 0.93
Penalty: l2, C: 1.0, tol: 0.0001, Accuracy: 0.92
Penalty: l2, C: 1.0, tol: 0.001, Accuracy: 0.92
Penalty: l2, C: 1.0, tol: 0.01, Accuracy: 0.92
Penalty: elasticnet, C: 0.001, tol: 0.0001, Accuracy: 0.51
Penalty: elasticnet, C: 0.001, tol: 0.001, Accuracy: 0.51
Penalty:

C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.0001, Accuracy: 0.83


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.001, Accuracy: 0.84
Penalty: None, C: 0.001, tol: 0.01, Accuracy: 0.85


...

Penalty: None, C: 0.1, tol: 0.0001, Accuracy: 0.83


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.1, tol: 0.001, Accuracy: 0.84
Penalty: None, C: 0.1, tol: 0.01, Accuracy: 0.85


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 1.0, tol: 0.0001, Accuracy: 0.83
Penalty: None, C: 1.0, tol: 0.001, Accuracy: 0.84
Penalty: None, C: 1.0, tol: 0.01, Accuracy: 0.85
Best Configuration: {'penalty': 'l2', 'C': 0.1, 'tol': 0.0001, 'l1_ratio': None}, Best Validation Accuracy: 0.93


In [41]:
best_model = LogisticRegression(penalty=best_params['penalty'], C=best_params['C'], tol=best_params['tol'], solver=solver, l1_ratio=best_params['l1_ratio'], max_iter=4000)
best_model.fit(X_train_embeddings, y_train)
y_test_pred = best_model.predict(X_test_embeddings)
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy ont the test set with the best configuration is: ", accuracy)

Accuracy ont the test set with the best configuration is:  0.865


In [42]:
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\iskak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iskak\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
X_train_embeddings = model.encode(train[:,0], batch_size=32, show_progress_bar=True)
X_val_embeddings = model.encode(val[:,0], batch_size=32, show_progress_bar=True)
X_test_embeddings = model.encode(test[:,0], batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [44]:
for penalty in penalties:
    for C in Cs:
        for tol in tols:
            # Elasticnet requires specifying l1_ratio; here, we use 0.5 as a default value.
            
            if penalty == 'elasticnet':
                l1_ratio = 0.5
            else:
                l1_ratio = None

            # Create and train the model
            model = LogisticRegression(penalty=penalty, C=C, tol=tol, solver=solver, l1_ratio=l1_ratio, max_iter=4000)
            try:
                model.fit(X_train_embeddings, y_train)
                # Predict on the validation set
                y_val_pred = model.predict(X_val_embeddings)
                # Calculate accuracy
                accuracy = accuracy_score(y_val, y_val_pred)
                print(f"Penalty: {penalty}, C: {C}, tol: {tol}, Accuracy: {accuracy}")

                # Update the best model if the current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'penalty': penalty, 'C': C, 'tol': tol, 'l1_ratio': l1_ratio}
            except ValueError as e:
                # Some configurations might be invalid depending on the solver, catch those errors
                print(f"Invalid configuration: Penalty: {penalty}, C: {C}, tol: {tol}, Error: {e}")

# Display the best configuration and its accuracy
print(f"Best Configuration: {best_params}, Best Validation Accuracy: {best_accuracy}")
best_model = LogisticRegression(penalty=best_params['penalty'], C=best_params['C'], tol=best_params['tol'], solver=solver, l1_ratio=best_params['l1_ratio'], max_iter=4000)
best_model.fit(X_train_embeddings, y_train)
y_test_pred = best_model.predict(X_test_embeddings)
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy ont the test set with the best configuration is: ", accuracy)

Penalty: l1, C: 0.001, tol: 0.0001, Accuracy: 0.49
Penalty: l1, C: 0.001, tol: 0.001, Accuracy: 0.49
Penalty: l1, C: 0.001, tol: 0.01, Accuracy: 0.51
Penalty: l1, C: 0.1, tol: 0.0001, Accuracy: 0.49
Penalty: l1, C: 0.1, tol: 0.001, Accuracy: 0.51
Penalty: l1, C: 0.1, tol: 0.01, Accuracy: 0.49
Penalty: l1, C: 1.0, tol: 0.0001, Accuracy: 0.96
Penalty: l1, C: 1.0, tol: 0.001, Accuracy: 0.96
Penalty: l1, C: 1.0, tol: 0.01, Accuracy: 0.97
Penalty: l2, C: 0.001, tol: 0.0001, Accuracy: 0.63
Penalty: l2, C: 0.001, tol: 0.001, Accuracy: 0.63
Penalty: l2, C: 0.001, tol: 0.01, Accuracy: 0.63
Penalty: l2, C: 0.1, tol: 0.0001, Accuracy: 0.97
Penalty: l2, C: 0.1, tol: 0.001, Accuracy: 0.97
Penalty: l2, C: 0.1, tol: 0.01, Accuracy: 0.97
Penalty: l2, C: 1.0, tol: 0.0001, Accuracy: 0.97
Penalty: l2, C: 1.0, tol: 0.001, Accuracy: 0.97
Penalty: l2, C: 1.0, tol: 0.01, Accuracy: 0.97
Penalty: elasticnet, C: 0.001, tol: 0.0001, Accuracy: 0.49
Penalty: elasticnet, C: 0.001, tol: 0.001, Accuracy: 0.51
Penalty

C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.0001, Accuracy: 0.97


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 0.001, tol: 0.001, Accuracy: 0.97


...

Penalty: None, C: 0.1, tol: 0.001, Accuracy: 0.97
Penalty: None, C: 0.1, tol: 0.01, Accuracy: 0.98


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Penalty: None, C: 1.0, tol: 0.0001, Accuracy: 0.97
Penalty: None, C: 1.0, tol: 0.001, Accuracy: 0.97
Penalty: None, C: 1.0, tol: 0.01, Accuracy: 0.98
Best Configuration: {'penalty': None, 'C': 0.001, 'tol': 0.01, 'l1_ratio': None}, Best Validation Accuracy: 0.98


C:\Users\iskak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Accuracy ont the test set with the best configuration is:  0.915


In [51]:
import stanza

stanza.download('en')  # Download English models
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment', tokenize_no_ssplit=True)

2024-03-03 12:44:21 INFO: Downloaded file to C:\Users\iskak\stanza_resources\resources.json
2024-03-03 12:44:21 INFO: Downloading default packages for language: en (English) ...


2024-03-03 12:48:07 INFO: Downloaded file to C:\Users\iskak\stanza_resources\en\default.zip
2024-03-03 12:48:10 INFO: Finished downloading models and saved to C:\Users\iskak\stanza_resources
2024-03-03 12:48:10 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-03-03 12:48:10 INFO: Downloaded file to C:\Users\iskak\stanza_resources\resources.json
2024-03-03 12:48:10 WARNING: Language en package default expects mwt, which has been added
2024-03-03 12:48:11 INFO: Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | combined       |
| mwt       | combined       |
| sentiment | sstplus_charlm |

2024-03-03 12:48:11 INFO: Using device: cpu
2024-03-03 12:48:11 INFO: Loading: tokenize
2024-03-03 12:48:11 INFO: Loading: mwt
2024-03-03 12:48:11 INFO: Loading: sentiment
2024-03-03 12:48:11 INFO: Done loading processors!


In [53]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment', batch_size=64)
val_sentiments = []

for text in val[:,0]:
    doc = nlp(text)
    sentiment = doc.sentences[0].sentiment
    val_sentiments.append(sentiment)

# Compute the most frequent sentiment in the validation set
val_sentiment_counts = {sentiment: val_sentiments.count(sentiment) for sentiment in val_sentiments}
most_frequent_sentiment = max(val_sentiment_counts, key=val_sentiment_counts.get)
print(f"Most frequent sentiment in the validation set: {most_frequent_sentiment}")

2024-03-03 12:51:52 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-03-03 12:51:52 INFO: Downloaded file to C:\Users\iskak\stanza_resources\resources.json
2024-03-03 12:51:52 WARNING: Language en package default expects mwt, which has been added
2024-03-03 12:51:52 INFO: Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | combined       |
| mwt       | combined       |
| sentiment | sstplus_charlm |

2024-03-03 12:51:52 INFO: Using device: cpu
2024-03-03 12:51:52 INFO: Loading: tokenize
2024-03-03 12:51:52 INFO: Loading: mwt
2024-03-03 12:51:52 INFO: Loading: sentiment
2024-03-03 12:51:53 INFO: Done loading processors!


Most frequent sentiment in the validation set: 0


In [60]:
test_sentiments = []

for text in test[:,0]:
    doc = nlp(text)
    sentiment = doc.sentences[0].sentiment
    if sentiment == 1:
        sentiment = most_frequent_sentiment  # Assign the most frequent sentiment from validation set to neutral texts
    if sentiment == 2:
        sentiment = 1
    test_sentiments.append(sentiment)

In [61]:
test_accuracy = accuracy_score(y_test, test_sentiments)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.955
